In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from hypso import Hypso1, Hypso2
sys.path.append(os.path.abspath("D:/Hierarchical Unmixing Label"))
import machi.base as ac

### FUNCTIONS

In [ ]:
#Functions
def plot_cm_rgb_composite(cm, title="", red_band_index=69, green_band_index=46, blue_band_index=26, aspect=0.1, figsize=(10, 10), height=1092, contrast_enhancement=False):
    """
    Create and plot an RGB composite image from a hyperspectral cube.
    
    Parameters:
    - cm: Input hyperspectral data cube or array (can be flattened (h*w, n_bands) or 3D (h, w, n_bands))
    - title: Optional title for the plot
    - red_band_index: Index of the band to use for red channel (default: 69)
    - green_band_index: Index of the band to use for green channel (default: 46)
    - blue_band_index: Index of the band to use for blue channel (default: 26)
    - aspect: Aspect ratio for the plot (default: 0.1)
    - figsize: Figure size as tuple (width, height) in inches (default: (10, 10))
    - height: Height of the image when reshaping from flattened data (default: 1092)
    
    Returns:
    - rgb_image: The processed RGB image
    """
    # Check if input is already a 3D cube or needs reshaping
    if len(cm.shape) == 2:  # Flattened data (h*w, n_bands)
        width = cm.shape[0] // height
        data_cube = cm.reshape(width, height, cm.shape[1])
        data_cube = np.transpose(data_cube, (0, 1, 2))  # Ensure correct orientation
    elif len(cm.shape) == 3:  # Already a cube (h, w, n_bands)
        data_cube = cm
    else:
        raise ValueError(f"Unexpected input shape: {cm.shape}. Expected 2D or 3D array.")
    
    # Extract the specified bands for RGB channels
    red_band = data_cube[:, :, red_band_index]
    green_band = data_cube[:, :, green_band_index]
    blue_band = data_cube[:, :, blue_band_index]

    # Stack the bands to create an RGB image
    rgb_image = np.stack((red_band, green_band, blue_band), axis=-1)
    
    # Process data for better visualization
    # Replace NaN values with 0
    rgb_image[np.isnan(rgb_image)] = 0

    # Apply normalization to each channel
    for i in range(3):
        channel = rgb_image[:,:,i]
        
        # Always use min-max normalization regardless of value range
        min_val = np.nanmin(channel)
        max_val = np.nanmax(channel)
        
        if max_val > min_val:  # Avoid division by zero
            # Normalize to [0,1] range
            channel = (channel - min_val) / (max_val - min_val)
            
            # Apply contrast enhancement if requested
            if contrast_enhancement and np.any(channel > 0):
                # Only enhance contrast if we have enough non-zero values
                non_zero_values = channel[channel > 0]
                if len(non_zero_values) > 10:  # Arbitrary threshold
                    percentiles = np.nanpercentile(channel, [2, 98])
                    p_low, p_high = percentiles[0], percentiles[1]
                    if p_high > p_low:
                        channel = np.clip(channel, p_low, p_high)
                        channel = (channel - p_low) / (p_high - p_low)
            
            rgb_image[:,:,i] = channel
    
    # Final normalization and cleanup
    rgb_image = np.clip(rgb_image, 0, 1)
    
    # Rotate for proper orientation
    rgb_image = np.rot90(rgb_image)
    
    # Create and display the plot
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    if title:
        plt.title(title)
    plt.imshow(rgb_image, aspect=aspect)
    plt.axis('off')  # Hide axes for cleaner visualization
    
    return rgb_image

def create_rgb_composite_from_satobj(satobj, title="", cube="L1A", red_band_index=69, green_band_index=46, blue_band_index=26, aspect=0.1, figsize=(10, 10), display=True, contrast_enhancement=False ):
    """
    Create an RGB composite image from a satellite object (satobj).

    Parameters:
    - satobj: An instance of the satellite object containing the image data.
    - red_band_index: The index of the band to use for the red channel (default: 69).
    - green_band_index: The index of the band to use for the green channel (default: 46).
    - blue_band_index: The index of the band to use for the blue channel (default: 26).
    - aspect: Aspect ratio for the plot (default: 0.1).
    - figsize: Figure size as tuple (width, height) in inches (default: (10, 10)).
    - display: Whether to display the image (default: True).
    - cube: Which data cube to use ("L1A", "L1B", "L1C", or "L1D") (default: "L1A").

    Returns:
    - rgb_image: A 3D numpy array representing the RGB composite image.
    """
    # Get the data cube from the satellite object
    if cube =="L1A":
        data_cube = satobj.l1a_cube  
    elif cube =="L1B":
        data_cube = satobj.l1b_cube  
    elif cube =="L1C":
        data_cube = satobj.l1c_cube 
    elif cube =="L1D":
        data_cube = satobj.l1d_cube
    else:
        print("cube not valid, defaulting tp L1A")
        data_cube = satobj.l1a_cube     
    
    # Extract the specified bands for RGB channels
    red_band = data_cube[:, :, red_band_index]
    green_band = data_cube[:, :, green_band_index]
    blue_band = data_cube[:, :, blue_band_index]

    # Stack the bands to create an RGB image
    rgb_image = np.stack((red_band, green_band, blue_band), axis=-1)

    # Handle NaN values
    rgb_image[np.isnan(rgb_image)] = 0

    # Apply normalization to each channel
    for i in range(3):
        channel = rgb_image[:,:,i]
        
        # Always use min-max normalization regardless of value range
        min_val = np.nanmin(channel)
        max_val = np.nanmax(channel)
        
        if max_val > min_val:  # Avoid division by zero
            # Normalize to [0,1] range
            channel = (channel - min_val) / (max_val - min_val)
            
            # Apply contrast enhancement if requested
            if contrast_enhancement and np.any(channel > 0):
                # Only enhance contrast if we have enough non-zero values
                non_zero_values = channel[channel > 0]
                if len(non_zero_values) > 10:  # Arbitrary threshold
                    percentiles = np.nanpercentile(channel, [2, 98])
                    p_low, p_high = percentiles[0], percentiles[1]
                    if p_high > p_low:
                        channel = np.clip(channel, p_low, p_high)
                        channel = (channel - p_low) / (p_high - p_low)
            
            rgb_image[:,:,i] = channel
    
    # Final normalization and cleanup
    rgb_image = np.clip(rgb_image, 0, 1)
    
    # Create properly oriented image for display
    display_image = np.rot90(rgb_image)
    
    # Display the image if requested
    if display:
        fig, ax = plt.subplots(figsize=figsize)
        ax.imshow(display_image, aspect=aspect, interpolation='nearest')
        ax.set_title(title+" "+cube)
        ax.axis('off')
        plt.show()

    return rgb_image

def hypso_MACHI_Concatenate(hypso_file_path, cube_combined, saturated_combined, verbose=True, correction="L1D", hypso=1, spectra=112):

    image_name = os.path.splitext(os.path.basename(hypso_file_path))[0]
    if verbose:
        print(f"Processing {image_name}")

    correction_with_spectra = f"{correction}_{spectra}"
    
    # load and calibrate the hypso image, e.g. convert to radiance
    if hypso==2:
        satobj = Hypso2(path=hypso_file_path, verbose=True)
    else:
        satobj = Hypso1(path=hypso_file_path, verbose=True)

    # Identify saturated pixels (values > 35000 in raw data)
    maxes = np.max(np.array(satobj.l1a_cube), axis = -1) #rawcube is not an attribute of the Hypso 2.0+ object  
    saturated = maxes > 35000
    saturated_flat = saturated.reshape(-1)
    saturated_flat = np.asarray(saturated_flat, dtype=bool)

    if correction == "L1A":
        data = satobj.l1a_cube
    elif correction == "L1B":
        satobj.generate_l1b_cube()
        data = satobj.l1b_cube
    elif correction == "L1C":
        satobj.generate_l1b_cube()
        satobj.generate_l1c_cube()
        data = satobj.l1c_cube
    else:
        satobj.generate_l1b_cube()
        satobj.generate_l1c_cube()
        satobj.generate_l1d_cube()
        data = satobj.l1d_cube

   # Extract spectral bands 
    if spectra==120:
        cube=np.array(data)
        c = cube.reshape(-1,spectra)
    else:
        spectra=112
        cube=np.array(data)[:,:,6:118]
        c = cube.reshape(-1,spectra)
    
    if verbose:
        print("saturated.shape: ",saturated.shape)
        print("cube.shape: ",cube.shape)
        print("c.shape: ",c.shape)

    T, S, objs = ac.atm_correction(c, solar=np.ones(spectra), verbose=False, tol=0.01, est_min_R=0.05)
    cube_norm = (cube - S) /T
    c_norm = cube_norm.reshape(-1,spectra)
    
    mins = [np.nanmin(c_norm[:,i]) for i in range(spectra)] #when saturation sets to nan, the min is not defined and the following line will set the min to nan
    cm_norm = c_norm - 0.95*np.array(mins)
    
    # Handle NaN values
    cm_norm = np.nan_to_num(cm_norm, nan=0.0)
    
    cm_combined = np.concatenate((cm_norm, cube_combined), axis=0)
    saturated_output = np.concatenate((saturated_flat, saturated_combined), axis=0)
    saturated_output = np.asarray(saturated_output, dtype=bool)
    
    if verbose:
        print("cube_norm.shape: ",cube_norm.shape)
        print("c_norm.shape: ",c_norm.shape)
        print("cm_norm.shape: ",cm_norm.shape)
        print("cm_combined.shape: ",cm_combined.shape)
        print("saturated_flat.shape: ",saturated_flat.shape)
        print("saturated_output.shape: ",saturated_output.shape)

    # Reshape cm back to cube shape for visualization
    cm_cube = cm_norm.reshape(cube.shape)
    # Use the plot_cm_rgb_composite function for visualization
    rgb=plot_cm_rgb_composite(
        cm_cube, 
        title=f"{image_name} ("+correction_with_spectra+"MACHI)",
        red_band_index=69-6,  # Adjust indices to match the 6-118 band range
        green_band_index=46-6,
        blue_band_index=26-6,
        aspect=0.1,
        figsize=(10, 10)
    )
    # Save outputs
    output_dir = r'D:\Hierarchical Unmixing Label\hUH\images'
    plt.savefig(os.path.join(output_dir, f'{image_name}_rgb_composite_{correction_with_spectra}_MACHI.png'))
    np.save(os.path.join(output_dir, f'{image_name}_cm_{correction_with_spectra}_MACHI.npy'), cm_norm)
    np.save(os.path.join(output_dir, f'{image_name}_saturated_{correction_with_spectra}_MACHI.npy'), saturated_flat)
    if verbose:
        print(f"File saved to: {os.path.join(output_dir, f'{image_name}_cm_{correction_with_spectra}_MACHI.npy')}")
        print("-"*50+'\n')
    return cm_combined, saturated_output, cm_norm, satobj

def hypso_Concatenate(hypso_file_path, cm_combined_input, saturated_combined_input, verbose=True, correction="L1A", hypso=1, spectra=120):
    """
    Process Hypso L1B data, normalize it, and concatenate with existing data.
    Uses a single, consistent normalization approach.
    
    Parameters:
    -----------
    hypso_file_path : str
        Path to the Hypso L1A file
    cm_combined_input : numpy.ndarray
        Existing combined cube data to append to
    saturated_combined_input : numpy.ndarray
        Existing combined saturation mask to append to
    verbose : bool, optional
        Whether to print processing information (default: True)
        
    Returns:
    --------
    tuple
        (combined data, combined saturation mask, normalized data, Hypso object)
    """
    # Extract image name from file path
    image_name = os.path.splitext(os.path.basename(hypso_file_path))[0]
    if verbose:
        print(f"Processing {image_name} using {correction}")
    
    # Load and calibrate the hypso image
    if hypso==2:
        satobj = Hypso2(path=hypso_file_path, verbose=True)
    else:
        satobj = Hypso1(path=hypso_file_path, verbose=True)
    # Identify saturated pixels (values > 35000 in raw data)
    maxes = np.max(np.array(satobj.l1a_cube), axis=-1)
    saturated = maxes > 35000
    
    if correction == "L1A":
        data = satobj.l1a_cube
    elif correction == "L1B":
        satobj.generate_l1b_cube()
        data = satobj.l1b_cube
    elif correction == "L1C":
        satobj.generate_l1c_cube()
        data = satobj.l1c_cube
    elif correction == "L1D":
        satobj.generate_l1d_cube()
        data = satobj.l1d_cube
    else:
        print("correction valid, defaulting to L1B")
        correction="L1B"
        data = satobj.l1b_cube

    # Extract spectral bands 6-118
    if spectra==112:
        cube=np.array(data)[:,:,6:118]
        c = cube.reshape(-1,112)
    elif spectra==120:
        cube=np.array(data)
        c = cube.reshape(-1,120)
    # Print information about the image data values
    # Combine correction level with spectral band count to create a complete correction identifier
    correction_with_spectra = f"{correction}_{spectra}"
    
  
    if verbose:
        pass
    
    # if not correction == "L1D":
    #     # Normalize the data to 0-1 range for atmospheric correction
    #     # First, find the min and max values for each band
    #     band_mins = np.nanmin(c, axis=0)
    #     band_maxs = np.nanmax(c, axis=0)
        
    #     # Check for bands where min and max are the same to avoid division by zero
    #     valid_bands = (band_maxs - band_mins) > 0
        
    #     # Create a normalized copy of the data
    #     c_normalized = np.zeros_like(c, dtype=np.float32)
        
    #     # Normalize each band individually
    #     for i in range(c.shape[1]):
    #         if valid_bands[i]:
    #             c_normalized[:, i] = (c[:, i] - band_mins[i]) / (band_maxs[i] - band_mins[i])
    #         else:
    #             # For bands with no variation, set to a small value to avoid issues
    #             c_normalized[:, i] = 0.01
        
    #     # Ensure all values are strictly within 0-1 range
    #     c_normalized = np.clip(c_normalized, 0, 1)
        
    #     # Use the normalized data for atmospheric correction
    #     c = c_normalized
    # Reshape for processing
    saturated_flat = saturated.reshape(-1)
    saturated_flat = np.asarray(saturated_flat, dtype=bool)
    
    # # Calculate minimum values for each band
    # mins = [c[:,i].min() for i in range(112)]
    # # Apply normalization: subtract 95% of minimum value for each band
    # cm = c - 0.95*np.array(mins)
    # # Handle NaN values
    # cm = np.nan_to_num(cm, nan=0.0)
    cm=c
    
    # Combine with existing data
    cm_combined = np.concatenate((cm, cm_combined_input), axis=0)
    saturated_combined = np.concatenate((saturated_flat, saturated_combined_input), axis=0)
    saturated_combined = np.asarray(saturated_combined, dtype=bool)
    
    if verbose:
        pass

    # Reshape cm back to cube shape for visualization
    cm_cube = cm.reshape(cube.shape)
    # Use the plot_cm_rgb_composite function for visualization
    rgb=plot_cm_rgb_composite(
        cm_cube, 
        title=f"{image_name} "+correction_with_spectra,
        red_band_index=69-6,  # Adjust indices to match the 6-118 band range
        green_band_index=46-6,
        blue_band_index=26-6,
        aspect=0.1,
        figsize=(10, 10)
    )
    # Print information about the image data values
    
    # Save outputs
    output_dir = r'D:\Hierarchical Unmixing Label\hUH\images'
    # Fix the file path by properly joining the strings
    plt.savefig(os.path.join(output_dir, f'{image_name}_rgb_composite_{correction_with_spectra}.png'))
    np.save(os.path.join(output_dir, f'{image_name}_cm_{correction_with_spectra}.npy'), cm)
    np.save(os.path.join(output_dir, f'{image_name}_saturated_{correction_with_spectra}.npy'), saturated_flat)
    print("-"*50+'\n')

    return cm_combined, saturated_combined, cm, satobj

def print_image_info(image_data, image_name="Image"):
    """
    Print statistics about an image file
    
    Parameters:
    -----------
    image_data : numpy.ndarray
        The image data to analyze
    image_name : str
        Name of the image for display purposes
    """
    # Basic statistics for the data
    print(f"{image_name} Statistics:")
    print(f"Shape: {image_data.shape}")
    print(f"Min value: {np.min(image_data)}")
    print(f"Max value: {np.max(image_data)}")
    print(f"Mean value: {np.mean(image_data)}")
    print(f"Median value: {np.median(image_data)}")
    print(f"Standard deviation: {np.std(image_data)}")
    
    # If the data has multiple bands/channels, analyze each band
    if len(image_data.shape) > 2:
        num_bands = image_data.shape[2] if len(image_data.shape) == 3 else image_data.shape[0]
        print(f"\nNumber of bands/channels: {num_bands}")
        
        # Print statistics for a few bands if there are many
        bands_to_show = min(5, num_bands)
        for i in range(bands_to_show):
            if len(image_data.shape) == 3:
                band_data = image_data[:, :, i]
            else:
                band_data = image_data[i]
                
            print(f"\nBand {i} Statistics:")
            print(f"  Min value: {np.min(band_data)}")
            print(f"  Max value: {np.max(band_data)}")
            print(f"  Mean value: {np.mean(band_data)}")
            print(f"  Median value: {np.median(band_data)}")
            print(f"  Standard deviation: {np.std(band_data)}")
        
        if num_bands > bands_to_show:
            print(f"\n... and {num_bands - bands_to_show} more bands")


### COMBINE IMAGES

In [ ]:
#Get the data, 10 images, 15 min
machi_combined= np.empty((0,112))
machi_sat_combined = np.empty(0)

machi_combined, machi_sat_combined, machi_yucatan2_2025_02_06,            satobj_yucatan2_2025_02_06          =hypso_MACHI_Concatenate(r"D:\Downloads\yucatan2_2025-02-06T16-01-18Z-l1a.nc",          machi_combined, machi_sat_combined)
machi_combined, machi_sat_combined, machi_kemigawa_2024_12_17,            satobj_kemigawa_2024_12_17          =hypso_MACHI_Concatenate(r"D:\Downloads\kemigawa_2024-12-17T01-01-32Z-l1a.nc",          machi_combined, machi_sat_combined)
machi_combined, machi_sat_combined, machi_chapala_2025_02_24,             satobj_chapala_2025_02_24           =hypso_MACHI_Concatenate(r"D:\Downloads\chapala_2025-02-24T16-52-47Z-l1a.nc",           machi_combined, machi_sat_combined)
machi_combined, machi_sat_combined, machi_grizzlybay_2025_01_27,          satobj_grizzlybay_2025_01_27        =hypso_MACHI_Concatenate(r"D:\Downloads\grizzlybay_2025-01-27T18-19-56Z-l1a.nc",        machi_combined, machi_sat_combined)
machi_combined, machi_sat_combined, machi_victoriaLand_2025_02_07,        satobj_victoriaLand_2025_02_07      =hypso_MACHI_Concatenate(r"D:\Downloads\victoriaLand_2025-02-07T20-35-33Z-l1a.nc",      machi_combined, machi_sat_combined)
machi_combined, machi_sat_combined, machi_catala_2025_01_28,              satobj_catala_2025_01_28            =hypso_MACHI_Concatenate(r"D:\Downloads\catala_2025-01-28T19-17-32Z-l1a.nc",            machi_combined, machi_sat_combined)
machi_combined, machi_sat_combined, machi_khnifiss_2025_02_12,            satobj_khnifiss_2025_02_12          =hypso_MACHI_Concatenate(r"D:\Downloads\khnifiss_2025-02-12T11-05-35Z-l1a.nc",          machi_combined, machi_sat_combined)
machi_combined, machi_sat_combined, machi_menindee_2025_02_18,            satobj_menindee_2025_02_18          =hypso_MACHI_Concatenate(r"D:\Downloads\menindee_2025-02-18T00-10-42Z-l1a.nc",          machi_combined, machi_sat_combined)
machi_combined, machi_sat_combined, machi_falklandsatlantic_2024_12_18,   satobj_falklandsatlantic_2024_12_18 =hypso_MACHI_Concatenate(r"D:\Downloads\falklandsatlantic_2024-12-18T13-25-18Z-l1a.nc", machi_combined, machi_sat_combined)
machi_combined, machi_sat_combined, machi_tampa_2024_11_12,               satobj_tampa_2024_11_12             =hypso_MACHI_Concatenate(r"D:\Downloads\tampa_2024-11-12T15-31-55Z-l1a.nc",             machi_combined, machi_sat_combined)
# machi_combined, machi_sat_combined, machi_aquawatchmoreton_2024_09_02,    satobj_aquawatchmoreton_2024_09_02  =hypso_MACHI_Concatenate(r"D:\Downloads\aquawatchmoreton_2024-09-02T23-10-28Z-l1a.nc",  machi_combined, machi_sat_combined)
# machi_combined, machi_sat_combined, machi_mjosa_2025_02_11,               satobj_mjosa_2025_02_11             =hypso_MACHI_Concatenate(r"D:\Downloads\mjosa_2025-02-11T09-56-45Z-l1a.nc",             machi_combined, machi_sat_combined)

In [ ]:
#Get the data, 10 images, 3 min(l1A & l1B) 10m(l1a - l1d)
l1a_combined= np.empty((0,120))
l1a_sat_combined = np.empty(0)

l1a_combined, l1a_sat_combined, l1a_yucatan2_2025_02_06,            satobj_l1a_yucatan2_2025_02_06          =hypso_Concatenate(r"D:\Downloads\yucatan2_2025-02-06T16-01-18Z-l1a.nc",          l1a_combined, l1a_sat_combined)
l1a_combined, l1a_sat_combined, l1a_kemigawa_2024_12_17,            satobj_l1a_kemigawa_2024_12_17          =hypso_Concatenate(r"D:\Downloads\kemigawa_2024-12-17T01-01-32Z-l1a.nc",          l1a_combined, l1a_sat_combined)
l1a_combined, l1a_sat_combined, l1a_chapala_2025_02_24,             satobj_l1a_chapala_2025_02_24           =hypso_Concatenate(r"D:\Downloads\chapala_2025-02-24T16-52-47Z-l1a.nc",           l1a_combined, l1a_sat_combined)
l1a_combined, l1a_sat_combined, l1a_grizzlybay_2025_01_27,          satobj_l1a_grizzlybay_2025_01_27        =hypso_Concatenate(r"D:\Downloads\grizzlybay_2025-01-27T18-19-56Z-l1a.nc",        l1a_combined, l1a_sat_combined)
l1a_combined, l1a_sat_combined, l1a_victoriaLand_2025_02_07,        satobj_l1a_victoriaLand_2025_02_07      =hypso_Concatenate(r"D:\Downloads\victoriaLand_2025-02-07T20-35-33Z-l1a.nc",      l1a_combined, l1a_sat_combined)
l1a_combined, l1a_sat_combined, l1a_catala_2025_01_28,              satobj_l1a_catala_2025_01_28            =hypso_Concatenate(r"D:\Downloads\catala_2025-01-28T19-17-32Z-l1a.nc",            l1a_combined, l1a_sat_combined)
l1a_combined, l1a_sat_combined, l1a_khnifiss_2025_02_12,            satobj_l1a_khnifiss_2025_02_12          =hypso_Concatenate(r"D:\Downloads\khnifiss_2025-02-12T11-05-35Z-l1a.nc",          l1a_combined, l1a_sat_combined)
l1a_combined, l1a_sat_combined, l1a_menindee_2025_02_18,            satobj_l1a_menindee_2025_02_18          =hypso_Concatenate(r"D:\Downloads\menindee_2025-02-18T00-10-42Z-l1a.nc",          l1a_combined, l1a_sat_combined)
l1a_combined, l1a_sat_combined, l1a_falklandsatlantic_2024_12_18,   satobj_l1a_falklandsatlantic_2024_12_18 =hypso_Concatenate(r"D:\Downloads\falklandsatlantic_2024-12-18T13-25-18Z-l1a.nc", l1a_combined, l1a_sat_combined)
l1a_combined, l1a_sat_combined, l1a_tampa_2024_11_12,               satobj_l1a_tampa_2024_11_12             =hypso_Concatenate(r"D:\Downloads\tampa_2024-11-12T15-31-55Z-l1a.nc",             l1a_combined, l1a_sat_combined)
# l1a_combined, l1a_sat_combined, l1a_aquawatchmoreton_2024_09_02,    satobj_l1a_aquawatchmoreton_2024_09_02  =hypso_Concatenate(r"D:\Downloads\aquawatchmoreton_2024-09-02T23-10-28Z-l1a.nc",  l1a_combined, l1a_sat_combined)
# l1a_combined, l1a_sat_combined, l1a_mjosa_2025_02_11,               satobj_l1a_mjosa_2025_02_11             =hypso_Concatenate(r"D:\Downloads\mjosa_2025-02-11T09-56-45Z-l1a.nc",             l1a_combined, l1a_sat_combined)

### SAVE

In [ ]:
np.save('D:\Hierarchical Unmixing Label\hUH\images\combined_10_L1D_112_MACHI.npy', machi_combined)
np.save('D:\Hierarchical Unmixing Label\hUH\images\saturated_combined_10_L1D_112_MACHI.npy', machi_sat_combined)

In [ ]:
np.save('D:\Hierarchical Unmixing Label\hUH\images\combined_10_L1A_120.npy', l1a_combined)
np.save('D:\Hierarchical Unmixing Label\hUH\images\saturated_combined_10_L1A_120.npy', l1a_sat_combined)

### LOAD

In [ ]:
machi_combined = np.load('D:\Hierarchical Unmixing Label\hUH\images\combined_10_v2_MACHI.npy')
machi_sat_combined = np.load('D:\Hierarchical Unmixing Label\hUH\images\saturated_combined_10_v2_MACHI.npy')

In [ ]:
machi_yucatan2_2025_02_06         = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\yucatan2_2025-02-06T16-01-18Z-l1a_cm_machi.npy')
machi_kemigawa_2024_12_17         = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\kemigawa_2024-12-17T01-01-32Z-l1a_cm_machi.npy')
machi_chapala_2025_02_24          = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\chapala_2025-02-24T16-52-47Z-l1a_cm_machi.npy')
machi_grizzlybay_2025_01_27       = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\grizzlybay_2025-01-27T18-19-56Z-l1a_cm_machi.npy')
machi_victoriaLand_2025_02_07     = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\victoriaLand_2025-02-07T20-35-33Z-l1a_cm_machi.npy')
machi_catala_2025_01_28           = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\catala_2025-01-28T19-17-32Z-l1a_cm_machi.npy')
machi_khnifiss_2025_02_12         = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\khnifiss_2025-02-12T11-05-35Z-l1a_cm_machi.npy')
machi_menindee_2025_02_18         = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\menindee_2025-02-18T00-10-42Z-l1a_cm_machi.npy')
machi_falklandsatlantic_2024_12_18= np.load(r'D:\Hierarchical Unmixing Label\hUH\images\falklandsatlantic_2024-12-18T13-25-18Z-l1a_cm_machi.npy')
machi_tampa_2024_11_12            = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\tampa_2024-11-12T15-31-55Z-l1a_cm_machi.npy')
# machi_aquawatchmoreton_2024_09_02 = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\aquawatchmoreton_2024-09-02T23-10-28Z-l1a_cm_machi.npy')
# machi_mjosa_2025_02_11            = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\mjosa_2025-02-11T09-56-45Z-l1a_cm_machi.npy')


In [ ]:
l1b_combined = np.load('D:\Hierarchical Unmixing Label\hUH\images\combined_10_v2_L1B.npy')
l1b_sat_combined = np.load('D:\Hierarchical Unmixing Label\hUH\images\saturated_combined_10_v2_L1B.npy')

In [ ]:
l1b_yucatan2_2025_02_06         = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\yucatan2_2025-02-06T16-01-18Z-l1a_cm_l1b.npy')
l1b_kemigawa_2024_12_17         = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\kemigawa_2024-12-17T01-01-32Z-l1a_cm_l1b.npy')
l1b_chapala_2025_02_24          = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\chapala_2025-02-24T16-52-47Z-l1a_cm_l1b.npy')
l1b_grizzlybay_2025_01_27       = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\grizzlybay_2025-01-27T18-19-56Z-l1a_cm_l1b.npy')
l1b_victoriaLand_2025_02_07     = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\victoriaLand_2025-02-07T20-35-33Z-l1a_cm_l1b.npy')
l1b_catala_2025_01_28           = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\catala_2025-01-28T19-17-32Z-l1a_cm_l1b.npy')
l1b_khnifiss_2025_02_12         = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\khnifiss_2025-02-12T11-05-35Z-l1a_cm_l1b.npy')
l1b_menindee_2025_02_18         = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\menindee_2025-02-18T00-10-42Z-l1a_cm_l1b.npy')
l1b_falklandsatlantic_2024_12_18= np.load(r'D:\Hierarchical Unmixing Label\hUH\images\falklandsatlantic_2024-12-18T13-25-18Z-l1a_cm_l1b.npy')
l1b_tampa_2024_11_12            = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\tampa_2024-11-12T15-31-55Z-l1a_cm_l1b.npy')
# l1b_aquawatchmoreton_2024_09_02 = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\aquawatchmoreton_2024-09-02T23-10-28Z-l1a_cm_l1b.npy')
# l1b_mjosa_2025_02_11            = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\mjosa_2025-02-11T09-56-45Z-l1a_cm_l1b.npy')

## PLOT

In [ ]:
rgb_maci_combined = plot_cm_rgb_composite(machi_combined, "machi_combined")

In [ ]:
rgb_machi_yucatan2_2025_02_06 = plot_cm_rgb_composite(machi_yucatan2_2025_02_06, "yucatan2_2025_02_06")
rgb_machi_kemigawa_2024_12_17 = plot_cm_rgb_composite(machi_kemigawa_2024_12_17, "kemigawa_2024_12_17")
rgb_machi_chapala_2025_02_24 = plot_cm_rgb_composite(machi_chapala_2025_02_24, "chapala_2025_02_24")
rgb_machi_grizzlybay_2025_01_27 = plot_cm_rgb_composite(machi_grizzlybay_2025_01_27, "grizzlybay_2025_01_27")
rgb_machi_victoriaLand_2025_02_07 = plot_cm_rgb_composite(machi_victoriaLand_2025_02_07, "victoriaLand_2025_02_07")
rgb_machi_catala_2025_01_28 = plot_cm_rgb_composite(machi_catala_2025_01_28, "catala_2025_01_28")
rgb_machi_khnifiss_2025_02_12 = plot_cm_rgb_composite(machi_khnifiss_2025_02_12, "khnifiss_2025_02_12")
rgb_machi_menindee_2025_02_18 = plot_cm_rgb_composite(machi_menindee_2025_02_18, "menindee_2025_02_18")
rgb_machi_falklandsatlantic_2024_12_18 = plot_cm_rgb_composite(machi_falklandsatlantic_2024_12_18, "falklandsatlantic_2024_12_18")
rgb_machi_tampa_2024_11_12 = plot_cm_rgb_composite(machi_tampa_2024_11_12, "tampa_2024_11_12")
#rgb_machi_aquawatchmoreton_2024_09_02 = plot_cm_rgb_composite(machi_aquawatchmoreton_2024_09_02, "aquawatchmoreton_2024_09_02") #Edge of Space
# rgb_machi_mjosa_2025_02_11 = plot_cm_rgb_composite(machi_mjosa_2025_02_11, "mjosa_2025_02_11") # directly into space

In [ ]:
rgb_l1b_combined = plot_cm_rgb_composite(l1b_combined, "l1b_combined")

In [ ]:
rgb_l1b_yucatan2_2025_02_06 = plot_cm_rgb_composite(l1b_yucatan2_2025_02_06, "yucatan2_2025_02_06")
rgb_l1b_kemigawa_2024_12_17 = plot_cm_rgb_composite(l1b_kemigawa_2024_12_17, "kemigawa_2024_12_17")
rgb_l1b_chapala_2025_02_24 = plot_cm_rgb_composite(l1b_chapala_2025_02_24, "chapala_2025_02_24")
rgb_l1b_grizzlybay_2025_01_27 = plot_cm_rgb_composite(l1b_grizzlybay_2025_01_27, "grizzlybay_2025_01_27")
rgb_l1b_victoriaLand_2025_02_07 = plot_cm_rgb_composite(l1b_victoriaLand_2025_02_07, "victoriaLand_2025_02_07")
rgb_l1b_catala_2025_01_28 = plot_cm_rgb_composite(l1b_catala_2025_01_28, "catala_2025_01_28")
rgb_l1b_khnifiss_2025_02_12 = plot_cm_rgb_composite(l1b_khnifiss_2025_02_12, "khnifiss_2025_02_12")
rgb_l1b_menindee_2025_02_18 = plot_cm_rgb_composite(l1b_menindee_2025_02_18, "menindee_2025_02_18")
rgb_l1b_falklandsatlantic_2024_12_18 = plot_cm_rgb_composite(l1b_falklandsatlantic_2024_12_18, "falklandsatlantic_2024_12_18")
rgb_l1b_tampa_2024_11_12 = plot_cm_rgb_composite(l1b_tampa_2024_11_12, "tampa_2024_11_12")
# rgb_l1b_aquawatchmoreton_2024_09_02 = plot_cm_rgb_composite(l1b_aquawatchmoreton_2024_09_02, "aquawatchmoreton_2024_09_02")
# rgb_l1b_mjosa_2025_02_11 = plot_cm_rgb_composite(l1b_mjosa_2025_02_11, "mjosa_2025_02_11")

### Satobj Plot

In [ ]:
s_rgb_yucatan2_2025_02_06 = create_rgb_composite_from_satobj(satobj_l1b_yucatan2_2025_02_06, "yucatan2_2025_02_06")
s_rgb_kemigawa_2024_12_17 = create_rgb_composite_from_satobj(satobj_l1b_kemigawa_2024_12_17, "kemigawa_2024_12_17")
s_rgb_chapala_2025_02_24 = create_rgb_composite_from_satobj(satobj_l1b_chapala_2025_02_24, "chapala_2025_02_24")
s_rgb_grizzlybay_2025_01_27 = create_rgb_composite_from_satobj(satobj_l1b_grizzlybay_2025_01_27, "grizzlybay_2025_01_27")
s_rgb_victoriaLand_2025_02_07 = create_rgb_composite_from_satobj(satobj_l1b_victoriaLand_2025_02_07, "victoriaLand_2025_02_07")
s_rgb_catala_2025_01_28 = create_rgb_composite_from_satobj(satobj_l1b_catala_2025_01_28, "catala_2025_01_28")
s_rgb_khnifiss_2025_02_12 = create_rgb_composite_from_satobj(satobj_l1b_khnifiss_2025_02_12, "khnifiss_2025_02_12")
s_rgb_menindee_2025_02_18 = create_rgb_composite_from_satobj(satobj_l1b_menindee_2025_02_18, "menindee_2025_02_18")
s_rgb_falklandsatlantic_2024_12_18 = create_rgb_composite_from_satobj(satobj_falklandsatlantic_2024_12_18, 'falklandsatlantic_2024_12_18')
s_rgb_tampa_2024_11_12= create_rgb_composite_from_satobj(satobj_tampa_2024_11_12, 'tampa_2024_11_12')
# s_rgb_aquawatchmoreton_2024_09_02 = create_rgb_composite_from_satobj(satobj_l1b_aquawatchmoreton_2024_09_02, "aquawatchmoreton_2024_09_02")
# s_rgb_mjosa_2025_02_11 = create_rgb_composite_from_satobj(satobj_l1b_mjosa_2025_02_11, "mjosa_2025_02_11")

### Compare plots

In [ ]:
rgb_menindee_2025_02_18     = plot_cm_rgb_composite(machi_menindee_2025_02_18, "menindee_2025_02_18 MACHI + PROCESSING", contrast_enhancement=False)
rgb_l1b_menindee_2025_02_18 = plot_cm_rgb_composite(l1b_menindee_2025_02_18, "menindee_2025_02_18 L1B + PROCESSING", contrast_enhancement=False)
l1a_rgb_menindee_2025_02_18 = create_rgb_composite_from_satobj(satobj_l1b_menindee_2025_02_18, "menindee_2025_02_18", cube="L1A", contrast_enhancement=False)
l1b_rgb_menindee_2025_02_18 = create_rgb_composite_from_satobj(satobj_l1b_menindee_2025_02_18, "menindee_2025_02_18", cube="L1B", contrast_enhancement=False)
l1c_rgb_menindee_2025_02_18 = create_rgb_composite_from_satobj(satobj_l1b_menindee_2025_02_18, "menindee_2025_02_18", cube="L1C", contrast_enhancement=False)
l1d_rgb_menindee_2025_02_18 = create_rgb_composite_from_satobj(satobj_l1b_menindee_2025_02_18, "menindee_2025_02_18", cube="L1D", contrast_enhancement=False)

In [ ]:
rgb_victoriaLand_2025_02_07     = plot_cm_rgb_composite(machi_victoriaLand_2025_02_07, "victoriaLand_2025_02_07 MACHI + PROCESSING")
rgb_l1b_victoriaLand_2025_02_07 = plot_cm_rgb_composite(l1b_victoriaLand_2025_02_07, "victoriaLand_2025_02_07 L1B + PROCESSING", contrast_enhancement=True)
l1a_rgb_victoriaLand_2025_02_07 = create_rgb_composite_from_satobj(satobj_l1b_victoriaLand_2025_02_07, "victoriaLand_2025_02_07", cube="L1A")
l1b_rgb_victoriaLand_2025_02_07 = create_rgb_composite_from_satobj(satobj_l1b_victoriaLand_2025_02_07, "victoriaLand_2025_02_07", cube="L1B")
l1c_rgb_victoriaLand_2025_02_07 = create_rgb_composite_from_satobj(satobj_l1b_victoriaLand_2025_02_07, "victoriaLand_2025_02_07", cube="L1C")
l1d_rgb_victoriaLand_2025_02_07 = create_rgb_composite_from_satobj(satobj_l1b_victoriaLand_2025_02_07, "victoriaLand_2025_02_07", cube="L1D")

## TEST

In [ ]:
#Get the data, 10 images, 15 min
l1b_combined= np.empty((0,120))
l1b_sat_combined = np.empty(0)

l1b_combined, l1b_sat_combined, aregantsea2_2025_03_11, satobj_aregantsea2_2025_03_11=hypso_Concatenate(r"D:\Downloads\aregantsea2_2025-03-11T08-12-43Z-l1a.nc",l1b_combined, l1b_sat_combined, correction="L1A", spectra=120)
l1b_combined, l1b_sat_combined, losmanzanosfire_2025_03_11, losmanzanosfire_2025_03_11_satobj=hypso_Concatenate(r"D:\Downloads\losmanzanosfire_2025-03-11T14-56-42Z-l1a.nc",l1b_combined, l1b_sat_combined, hypso=2, correction="L1A", spectra=120)

In [ ]:
#Get the data, 10 images, 15 min
machi_combined= np.empty((0,112))
machi_sat_combined = np.empty(0)

machi_combined, machi_sat_combined, losmanzanosfire_2025_03_11, losmanzanosfire_2025_03_11_satobj=hypso_MACHI_Concatenate(r"D:\Downloads\losmanzanosfire_2025-03-11T14-56-42Z-l1a.nc",machi_combined, machi_sat_combined, hypso=2)

In [ ]:
# Define a function to print information about values in an image file


# Example usage
print_image_info(aregantsea2_2025_03_11)
print_image_info(losmanzanosfire_2025_03_11)


# Notes
* Can run Machi on the large concatenated cube instead of each imgage before concatenating.
* save the the cm and saturation array as a file and load it instead of running the function again.
* Setting neighbor weight takes a lot of RAM (24GB)
* Endmember count does not affect ram usage
* reducing endmember by half sped up calculations by 50%
* 10 images, 16 endmembers and 10 runs took 60 minutes for one equlibiration
* 10 images, 8 endmembers and 10 runs took 45 minutes for one equlibiration
* sparsity sweep takes a lot of time (200 to 300 minutes)
* 10 images, 8 endmembers and 10 runs took 700 min for full network
## Labels
### dehx_10img_8end_10runs_fast_stabelized_aa_FINAL.h5
### lacrau image
* 1000 is oversaturated(clounds and snow)
* 0110 coastline?
* 0101 water (water 2 less spead)
* 0100 water (ocean mainly)
* 0011 rock, sand, urban (more urban leaning?)
* 0010 rock, sand, urban
* 0001 forest 2
* 0000 forest
### dehx_10img_8end_10runs_fast_stabelized_aa_FINAL.h5
### Kemigawa
* 1000 clouds
* 0110 water(coast?, freshwater?) 
* 0101 water2 
* 0100 water
* 0011 rock stone clud edge
* 0010 stone, rock, urban
* 0001 cloud edge, water edge forest edge
* 0000 Forest
### dehx_10img_8end_10runs_fast_stabelized_aa_FINAL.h5
### Kemigawa
* 1000 oversaturated
* 0110 water shallow
* 0101 water deep
* 0100 water deep coast
* 0011 urban, coast sand?
* 0010 rock, sand mountain, desert
* 0001 seperation between forest and rock(thin forest?)
* 0000 forest
###  GB RAM Cost 
* running AI env              1.3GB --baseline
* +all imports                1.4GB (+0.1GB)
* +Functions                  1.5GB (+0.1GB)
* +hypso_MACHI_Concatenate    11.5GB (+10GB)
* -2GB for the each hypso object
* +save the cm_combined and saturated_combined array to a file
* +Loading the cm_combined and saturated_combined array from a file (6.0GB)
* +Loading the cm_norm arrays (6.0GB)